In [1]:
# Generate graphs for demonstration purposes
import utils.graph_generators as gen
import torch
import dgl

grids = gen.make_grid_graphs()
lobsters = gen.make_lobster_graphs()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
grids = [dgl.DGLGraph(g).to(device) for g in grids] # Convert graphs to DGL from NetworkX
lobsters = [dgl.DGLGraph(g).to(device) for g in lobsters] # Convert graphs to DGL from NetworkX

c:\Users\cs843\AppData\Local\Programs\Python\Python310\lib\site-packages\dgl\heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


In [2]:
# Compute all GNN-based metrics at once
from evaluation.evaluator import Evaluator
evaluator = Evaluator(device=device)
evaluator.evaluate_all(generated_dataset=grids, reference_dataset=lobsters)


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))


{'activations_time': 0.09900069236755371,
 'fid': 5126.874207380354,
 'fid_time': 0.10399937629699707,
 'kid': 613081.2,
 'kid_time': 8.362998247146606,
 'precision': 0.0,
 'recall': 0.0,
 'f1_pr': 1e-05,
 'precision_time': 0.12754583358764648,
 'recall_time': 0.12754583358764648,
 'f1_pr_time': 0.12754583358764648,
 'density': 0.0,
 'coverage': 0.0,
 'f1_dc': 1e-05,
 'density_time': 0.1172800064086914,
 'coverage_time': 0.1172800064086914,
 'f1_dc_time': 0.1172800064086914,
 'mmd_rbf': 1.3138309940695763,
 'mmd_rbf_time': 0.1250011920928955,
 'mmd_linear': 4773.8984,
 'mmd_linear_time': 0.09900069236755371}

In [3]:
# Alternatively, compute a single GNN-based metric. See evaluation/gin_evaluation.py for other metrics.
from evaluation.gin_evaluation import load_feature_extractor, MMDEvaluation

# Can tweak GIN hyperparameters, however defaults are set to our recommendations
gin = load_feature_extractor(device=device)
# Can tweak hyperparameters of MMD RBF, however defaults are set to our recommendations
mmd_eval = MMDEvaluation(gin)
result, time = mmd_eval.evaluate(generated_dataset=grids, reference_dataset=lobsters)
print('result: {}, time to compute: {:.3f}s'.format(result, time))

result: {'mmd_rbf': 1.3007222935557365}, time to compute: 0.115s


In [4]:
# Evaluation incorporating custom node/edge features.
# If no node features are provided, the default is degree features
# expressed as an integer.

node_feat_loc = 'attr'
edge_feat_loc = 'attr'
node_feat_dim = 10
edge_feat_dim = 5

# Create random node/edge features
for g in grids:
    num_nodes = g.number_of_nodes()
    g.ndata[node_feat_loc] = torch.randn(num_nodes, node_feat_dim).to(g.device)
    num_edges = g.number_of_edges()
    g.edata[edge_feat_loc] = torch.randn(num_edges, edge_feat_dim).to(g.device)
    
for g in lobsters:
    num_nodes = g.number_of_nodes()
    g.ndata[node_feat_loc] = torch.randn(num_nodes, node_feat_dim).to(g.device)
    num_edges = g.number_of_edges()
    g.edata[edge_feat_loc] = torch.randn(num_edges, edge_feat_dim).to(g.device)

# Load GIN
gin = load_feature_extractor(
    device=device, input_dim=node_feat_dim, edge_feat_dim=edge_feat_dim,
    node_feat_loc=node_feat_loc, edge_feat_loc=edge_feat_loc)

mmd_eval = MMDEvaluation(gin)
result, time = mmd_eval.evaluate(generated_dataset=lobsters, reference_dataset=grids)
print('result: {}, time to compute: {:.3f}s'.format(result, time))

result: {'mmd_rbf': 1.0303107798099518}, time to compute: 0.170s


In [5]:
# To quickly demonstrate sensitivity to node/edge features, let's create
# a copy of the grid graphs and slightly alter the node/edge feat distributions
import copy

result, time = mmd_eval.evaluate(generated_dataset=grids, reference_dataset=grids)
print('MMD RBF for grids with itself: {}'.format(result['mmd_rbf']))

grids_copy = [copy.deepcopy(g) for g in grids]
for g in grids_copy:
    g.ndata[node_feat_loc] *= 1.15
    g.edata[edge_feat_loc] *= 1.15
    
result, time = mmd_eval.evaluate(reference_dataset=grids, generated_dataset=grids_copy)
print('MMD RBF after altering node/edge feat. distributions: {}'.format(result['mmd_rbf']))

MMD RBF for grids with itself: 0
MMD RBF after altering node/edge feat. distributions: 0.033437252044677734
